In [14]:
import nsepy
from nsepy import get_history, get_index_pe_history
from datetime import date, datetime , timedelta
import pandas as pd
import random
import os
import ntpath
import glob
import time
import  warnings
from ipywidgets import IntProgress
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [15]:
data_path =  os.path.join(os.getcwd(),"nse_data\\data\\")
data_index = os.path.join(os.getcwd(),"nse_data\\index_data\\")
dump_path =  os.path.join(os.getcwd(),"nse_data\\dump\\")
meta_data =  os.path.join(os.getcwd(),"nse_data\\meta_data\\")
reports_path = os.path.join(os.getcwd(),"nse_data\\reports\\")
nse_indices = os.path.join(os.getcwd(),"nse_data\\nse_indices\\")
print(nse_indices)

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\nse_indices\


In [16]:
nifty500_df = pd.read_csv(os.path.join(nse_indices, "ind_nifty500list.csv"))
stocks_list = nifty500_df.Symbol.tolist()
len(stocks_list)

501

In [17]:
# stocks_list

In [18]:
stocks_list = list(dict.fromkeys(stocks_list))

add_stocks = ['VEDL', 'YESBANK']
skip_stocks = ['BBTC','IRB','M&MFIN','PVR', 'HATHWAY','HEXAWARE','']

stocks_list = stocks_list + add_stocks

stocks_list = sorted(set(stocks_list)-(set(skip_stocks)))
len(stocks_list)

# stocks_list = random.sample(stocks_list, len(stocks_list))


497

In [6]:
def get_server_last_update_date():
    date_for_update = get_history('TCS', date.today()-timedelta(days=10), date.today()+timedelta(days=1), index = False)
    date_for_update.reset_index(inplace = True)
    return date_for_update.Date.max()

server_date = get_server_last_update_date()
print("Server updated till: ",server_date)

start_date = date(2000, 1, 1)
end_date = date.today()

# start_date = date(2011, 5, 5)
# end_date = date(2011, 5, 9)



Server updated till:  2021-09-09


In [7]:
temp_date = date(2020, 9, 5)
x = get_history( 'DCBBANK', temp_date, end_date, index = False)
x.tail(2)

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2021-09-08,DCBBANK,EQ,96.4,96.5,97.05,94.0,94.50,94.7,95.75,488400,4.676424e+12,5869,211075,0.4322
2021-09-09,DCBBANK,EQ,94.7,94.0,94.75,93.6,93.85,93.9,94.13,490018,4.612596e+12,6365,161536,0.3297


In [8]:
# Function to get script data file path if present
def get_stock_file_path(script):
    for file in glob.glob(data_path + script + '_*.csv'):
        file = file.replace("\\", "/")
        return(file)


def get_stock_filename(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

# Function to check if the script data file already present or not: True or False


def stock_data_present(script):
    file = get_stock_file_path(script)
    if file:
        return True
    return False

# Function to get stock dataframe from script name


def stock_df_from_csv(script):
    file = get_stock_file_path(script)
    stock_df = pd.read_csv(file)
    return stock_df


def stock_start_end_date(script):
    stock_df = stock_df_from_csv(script)
    no_of_rows = stock_df.shape[0]
    start_date = pd.to_datetime(stock_df.Date).min()
    datetime_obj_s = start_date.to_pydatetime().date()
    last_update_date = pd.to_datetime(stock_df.Date).max()
    datetime_obj_l = last_update_date.to_pydatetime().date()
    return [no_of_rows, datetime_obj_s, datetime_obj_l]


# Function to save stocks data file in csv format
def stock_data_save(script, stock_df):
    update_date = pd.to_datetime(stock_df.Date).max()
    datetime_obj = update_date.to_pydatetime().date()
    script_file = data_path + script + '_' + \
        datetime_obj.strftime('%d-%m-%Y') + '.csv'
    stock_df.to_csv(script_file, index=False)
    print(script_file, ' Saved to Disk')


# Function to generate a list of all stocks last updated date
def stock_data_table(stocks):
    for stock in stocks:
        if stock_data_present(stock):
            print('{:15s} Start Date: {}     Last Updated: {}     No of rows: {}'.format(stock,
                                                                                         stock_start_end_date(stock)[
                                                                                             1],
                                                                                         stock_start_end_date(stock)[
                                                                                             2],
                                                                                         stock_start_end_date(stock)[0]))
        else:
            print("{:15s} No Data found!!!".format(stock))


# Function to download the script data from NSE India
def script_download(script, start_date, end_date, index=False):
    print(script, ' Downloading from NSE India. \t', end='')
    stock_df = get_history(script, start_date, end_date, index)
    if stock_df.shape[0] > 0:
        print(script, ' Downloading complete')
        stock_df.reset_index(inplace=True)
        stock_data_save(script, stock_df)
#         update_date = pd.to_datetime(stock_df.Date).max()
#         datetime_obj = update_date.to_pydatetime().date()
#         script_file = data_path + script + '_' + datetime_obj.strftime('%d-%m-%Y') + '.csv'
#         stock_df.to_csv(script_file, index = False)
#         print(script_file, ' Saved to Disk')
    else:
        print(script, ' No Data Found!!!')


# Function to download data for new stocks from the stocks list
def stocks_downloader(stocks_list, start_date, end_date):
    present = 0
    download = 0
    for script in stocks_list:
        if stock_data_present(script):
            print('Data Already Present :', script)
            present = present+1
        else:
            script_download(script, start_date, end_date)
            download = download+1
    print("Total Present:", present, " Total downloaded:", download)


    
# Function to get a list of stocks to update
def scripts_to_update(stocks_list, server_date):
    update_list = []
    max_count = len(stocks_list)
    f = IntProgress(min=0, max=max_count, description='Getting List of Stocks to update:')
    display(f)

    for script in stocks_list:
        f.value += 1
        if stock_data_present(script):
            if stock_start_end_date(script)[2] < server_date:
                update_list.append(script)
    return update_list



# Function to update the script data from NSE India
def stock_data_update(script, end_date, index=False):
    start_date = stock_start_end_date(script)[2]
    print(script, ': Downloading data from ', start_date, 'to ', end_date)
    stock_df_update = get_history(script, start_date, end_date, index)
    if stock_df_update.shape[0] > 0:
        stock_df_update.reset_index(inplace=True)
        stock_df = stock_df_from_csv(script)
        stock_df = stock_df[:-1]
        stock_df = pd.concat([stock_df, stock_df_update], axis=0)
        file = get_stock_file_path(script)
        os.rename(file, dump_path + get_stock_filename(file))
        stock_data_save(script, stock_df)
    else:
        print(script, ' No Data Found!!! Try again later')

        
        
# Function to update all the stocks
def stocks_updater(stocks_list, server_date):
    stocks_list_update = scripts_to_update(stocks_list, server_date)
    
    f = IntProgress(min=0, max=len(stocks_list_update), description='Updating Stocks:',)
    display(f)

    for script in stocks_list_update:
        f.value += 1
        stock_data_update(script, server_date)
    print("All stocks updated")

In [9]:
# stocks_downloader(stocks_list, start_date, end_date)

In [10]:
import os, shutil
folder = dump_path
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [11]:
stocks_updater(stocks_list , server_date)

# x = get_history( 'LTTS', start_date, server_date, index= False)
# x.tail(5)

IntProgress(value=0, description='Getting List of Stocks to update:', max=497)

IntProgress(value=0, description='Updating Stocks:', max=497)

3MINDIA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\3MINDIA_09-09-2021.csv  Saved to Disk
AARTIIND : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\AARTIIND_09-09-2021.csv  Saved to Disk
AAVAS : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\AAVAS_09-09-2021.csv  Saved to Disk
ABB : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\ABB_09-09-2021.csv  Saved to Disk
ABBOTINDIA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\ABBOTINDIA_09-09-2021.csv  Saved to Disk
ABCAPITAL : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\ABCAPITAL_09-09-2021.csv  Saved to Disk
ABFRL : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anir

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BANKBARODA_09-09-2021.csv  Saved to Disk
BANKINDIA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BANKINDIA_09-09-2021.csv  Saved to Disk
BASF : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BASF_09-09-2021.csv  Saved to Disk
BATAINDIA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BATAINDIA_09-09-2021.csv  Saved to Disk
BAYERCROP : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BAYERCROP_09-09-2021.csv  Saved to Disk
BDL : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BDL_09-09-2021.csv  Saved to Disk
BEL : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\BEL_09-09-2021.

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\CROMPTON_09-09-2021.csv  Saved to Disk
CSBBANK : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\CSBBANK_09-09-2021.csv  Saved to Disk
CUB : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\CUB_09-09-2021.csv  Saved to Disk
CUMMINSIND : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\CUMMINSIND_09-09-2021.csv  Saved to Disk
CYIENT : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\CYIENT_09-09-2021.csv  Saved to Disk
DABUR : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\DABUR_09-09-2021.csv  Saved to Disk
DALBHARAT : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\DALBHARAT_09-09-2

...

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KAJARIACER_09-09-2021.csv  Saved to Disk
KALPATPOWR : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KALPATPOWR_09-09-2021.csv  Saved to Disk
KANSAINER : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KANSAINER_09-09-2021.csv  Saved to Disk
KARURVYSYA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KARURVYSYA_09-09-2021.csv  Saved to Disk
KEC : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KEC_09-09-2021.csv  Saved to Disk
KEI : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KEI_09-09-2021.csv  Saved to Disk
KNRCON : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\KNRCON_09-

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\MRF_09-09-2021.csv  Saved to Disk
MRPL : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\MRPL_09-09-2021.csv  Saved to Disk
MUTHOOTFIN : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\MUTHOOTFIN_09-09-2021.csv  Saved to Disk
NAM-INDIA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\NAM-INDIA_09-09-2021.csv  Saved to Disk
NATCOPHARM : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\NATCOPHARM_09-09-2021.csv  Saved to Disk
NATIONALUM : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\NATIONALUM_09-09-2021.csv  Saved to Disk
NAUKRI : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\N

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RAIN_09-09-2021.csv  Saved to Disk
RAJESHEXPO : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RAJESHEXPO_09-09-2021.csv  Saved to Disk
RALLIS : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RALLIS_09-09-2021.csv  Saved to Disk
RAMCOCEM : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RAMCOCEM_09-09-2021.csv  Saved to Disk
RATNAMANI : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RATNAMANI_09-09-2021.csv  Saved to Disk
RAYMOND : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RAYMOND_09-09-2021.csv  Saved to Disk
RBLBANK : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\RBLBA

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\SUPREMEIND_09-09-2021.csv  Saved to Disk
SUZLON : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\SUZLON_09-09-2021.csv  Saved to Disk
SWANENERGY : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\SWANENERGY_09-09-2021.csv  Saved to Disk
SWSOLAR : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\SWSOLAR_09-09-2021.csv  Saved to Disk
SYMPHONY : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\SYMPHONY_09-09-2021.csv  Saved to Disk
SYNGENE : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\SYNGENE_09-09-2021.csv  Saved to Disk
TAKE : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\TAKE_0

C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\VIPIND_09-09-2021.csv  Saved to Disk
VMART : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\VMART_09-09-2021.csv  Saved to Disk
VOLTAS : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\VOLTAS_09-09-2021.csv  Saved to Disk
VRLLOG : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\VRLLOG_09-09-2021.csv  Saved to Disk
VSTIND : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\VSTIND_09-09-2021.csv  Saved to Disk
VTL : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\VTL_09-09-2021.csv  Saved to Disk
WABCOINDIA : Downloading data from  2021-08-27 to  2021-09-09
C:\Users\Anirudh\Desktop\Technical_Analysis\nse_data\data\WABCOINDIA_09-09-2021.csv  S

In [12]:
len(stocks_list)

497

In [13]:
# Calculate Beta of stocks
from scipy.stats import linregress
linregress(nifty_returns, stock_returns)
round(linregress(nifty_returns, stock_returns).slope,4)

NameError: name 'nifty_returns' is not defined

In [ ]:
# Get Nifty Last 180 days returns
beta = []

nifty = get_history(symbol="NIFTY", start=date(2020,6,18), end=date(2021,6,18), index=True)
nifty["Prev Close"] = nifty['Close'].shift()
nifty = nifty[["Prev Close", "Close"]]
nifty["percentage_return"] = (nifty["Close"] / nifty["Prev Close"]) - 1
nifty = nifty.tail(180)
nifty_returns = nifty["percentage_return"].tolist()

# Get Stock Last 180 days returns
for stock in stocks_list:
    
    stock_df = stock_df_from_csv(stock)
        
    stock_df = stock_df.tail(180)
    stock_df = stock_df[["Date", "Prev Close", "Close"]]
    stock_df["percentage_return"] = (stock_df["Close"] / stock_df["Prev Close"]) - 1
    stock_returns = stock_df["percentage_return"].tolist()
    print(stock, round(linregress(nifty_returns, stock_returns).slope,4))
    beta.append({"Script":stock, "Beta": round(linregress(nifty_returns, stock_returns).slope,4)})
        

In [ ]:
beta

In [ ]:
beta_df = pd.DataFrame(beta)

In [ ]:
beta_df
beta_df.to_csv("Beta.csv", index = True)

In [81]:
# Get Nifty Last 180 days returns
beta = []

nifty = get_history(symbol="NIFTY", start=date(2000,1,1), end=date(2021,6,18), index=True)


In [84]:
nift_copy = nifty.copy()

In [85]:
nift_copy["Prev Close"] = nift_copy['Close'].shift()
nift_copy


,Open,High,Low,Close,Volume,Turnover,Prev Close
Date,,,,,,,
2000-01-03,1482.15,1592.90,1482.15,1592.20,25358322,8.841500e+09,NaN
2000-01-04,1594.40,1641.95,1594.40,1638.70,38787872,1.973690e+10,1592.20
2000-01-05,1634.55,1635.50,1555.05,1595.80,62153431,3.084790e+10,1638.70
2000-01-06,1595.80,1639.00,1595.80,1617.60,51272875,2.531180e+10,1595.80
2000-01-07,1616.60,1628.25,1597.20,1613.30,54315945,1.914630e+10,1617.60
...,...,...,...,...,...,...,...
2021-06-14,15791.40,15823.05,15606.50,15811.85,392862037,2.800150e+11,15799.35
2021-06-15,15866.95,15901.60,15842.40,15869.25,323262818,2.218126e+11,15811.85
2021-06-16,15847.50,15880.85,15742.60,15767.55,340191175,2.250595e+11,15869.25


In [ ]:
nifty["Prev Close"] = nifty['Close'].shift()
nifty = nifty[["Prev Close", "Close"]]
nifty["percentage_return"] = (nifty["Close"] / nifty["Prev Close"]) - 1
nifty = nifty.tail(180)
nifty_returns = nifty["percentage_return"].tolist()